### 평균가격(원)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 시각화 설정
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드
df = pd.read_csv(r".\data\train\train.csv", encoding="utf-8")

# 원본 데이터 복사
df_original = df.copy()

# 데이터 상태 확인
print("데이터 형태:", df.shape)
print("\n'시점' 열의 데이터 타입:", df['시점'].dtype)
print("\n'시점' 열의 고유값 (처음 5개):", df['시점'].unique()[:5])
print("\n'시점' 열의 데이터 예시 (처음 5개):")
print(df['시점'].head())

# '시점' 열 처리
def process_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    if isinstance(date_str, str):
        date_str = date_str.replace('상순', '05').replace('중순', '15').replace('하순', '25')
        return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')
    return pd.to_datetime(date_str, errors='coerce')

df['시점'] = df['시점'].apply(process_date)
df_original['시점'] = df_original['시점'].apply(process_date)

# '평균가격(원)'을 숫자형으로 변환하고 0을 NaN으로 처리
df['평균가격(원)'] = pd.to_numeric(df['평균가격(원)'], errors='coerce')
df['평균가격(원)'] = df['평균가격(원)'].replace(0, np.nan)

print("\n데이터 처리 후 '시점' 열의 데이터 타입:", df['시점'].dtype)
print("\n처리 후 '시점' 열의 데이터 예시 (처음 5개):")
print(df['시점'].head())

print("\n결측치 처리 전:")
print(df['평균가격(원)'].isnull().sum(), "개의 NaN 값")

# 품목별 품종 그룹 정의
crop_varieties = {
    '배추': ['배추', '쌈배추', '알배기배추', '얼갈이배추', '봄동배추', '절임배추'],
    '무': ['열무', '다발무', '무'],
    '양파': ['양파', '저장양파', '조생양파', '양파(햇)', '자주양파', '양파수입'],
    '사과': ['후지', '홍로', '쓰가루'],
    '배': ['신고', '원황'],
    '건고추': ['화건', '햇산양건', '양건'],
    '깐마늘': ['깐마늘(국산)'],
    '감자': ['감자 수미', '감자 수미(햇)', '감자', '감자 조풍', '감자 대지', '감자 두백', '감자 수미(저장)', '홍감자', '감자 수입'],
    '대파': ['대파(일반)', '실파', '깐쪽파', '쪽파', '대파수입'],
    '상추': ['청', '적']
}

# 건고추에 대한 특별 처리
ratio_yanggeon = df[df['품종명'] == '햇살양건']['평균가격(원)'].mean() / df[df['품종명'] == '양건']['평균가격(원)'].mean()
ratio_hwagun = df[df['품종명'] == '햇살화건']['평균가격(원)'].mean() / df[df['품종명'] == '화건']['평균가격(원)'].mean()

def fill_missing_chili(row):
    if pd.isnull(row['평균가격(원)']):
        if row['품종명'] == '햇살양건':
            yanggeon_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '양건')]['평균가격(원)'].values
            if len(yanggeon_value) > 0 and not pd.isnull(yanggeon_value[0]):
                return yanggeon_value[0] * ratio_yanggeon
        elif row['품종명'] == '햇살화건':
            hwagun_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '화건')]['평균가격(원)'].values
            if len(hwagun_value) > 0 and not pd.isnull(hwagun_value[0]):
                return hwagun_value[0] * ratio_hwagun
        elif row['품종명'] == '양건':
            sunyanggeon_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '햇살양건')]['평균가격(원)'].values
            if len(sunyanggeon_value) > 0 and not pd.isnull(sunyanggeon_value[0]):
                return sunyanggeon_value[0] / ratio_yanggeon
        elif row['품종명'] == '화건':
            sunhwagun_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '햇살화건')]['평균가격(원)'].values
            if len(sunhwagun_value) > 0 and not pd.isnull(sunhwagun_value[0]):
                return sunhwagun_value[0] / ratio_hwagun

        # 이전 및 이후 데이터 참조
        same_variety = df[df['품종명'] == row['품종명']].sort_values('시점')
        prev_value = same_variety[same_variety['시점'] < row['시점']]['평균가격(원)'].last_valid_index()
        next_value = same_variety[same_variety['시점'] > row['시점']]['평균가격(원)'].first_valid_index()
        
        if prev_value is not None and next_value is not None:
            return (df.loc[prev_value, '평균가격(원)'] + df.loc[next_value, '평균가격(원)']) / 2
        elif prev_value is not None:
            return df.loc[prev_value, '평균가격(원)']
        elif next_value is not None:
            return df.loc[next_value, '평균가격(원)']
    
    return row['평균가격(원)']

# 다른 작물에 대한 일반적인 결측치 처리 함수
def calculate_ratios(df, crop, varieties):
    ratios = {}
    base_variety = varieties[0]
    base_mean = df[df['품종명'] == base_variety]['평균가격(원)'].mean()
    
    for variety in varieties[1:]:
        variety_mean = df[df['품종명'] == variety]['평균가격(원)'].mean()
        if base_mean != 0 and not np.isnan(base_mean) and not np.isnan(variety_mean):
            ratios[variety] = variety_mean / base_mean
    
    return ratios, base_variety

def fill_missing_by_crop(row, crop, varieties, ratios, base_variety):
    if pd.isnull(row['평균가격(원)']) and row['품종명'] in varieties:
        if row['품종명'] == base_variety:
            other_varieties = [v for v in varieties if v != base_variety]
            for other in other_varieties:
                other_value = df[(df['시점'] == row['시점']) & (df['품종명'] == other)]['평균가격(원)'].values
                if len(other_value) > 0 and not pd.isnull(other_value[0]):
                    return other_value[0] / ratios.get(other, 1)
        else:
            base_value = df[(df['시점'] == row['시점']) & (df['품종명'] == base_variety)]['평균가격(원)'].values
            if len(base_value) > 0 and not pd.isnull(base_value[0]):
                return base_value[0] * ratios.get(row['품종명'], 1)

        # 이전 및 이후 데이터 참조
        same_variety = df[(df['품종명'] == row['품종명']) & (df['품목명'] == crop)].sort_values('시점')
        prev_value = same_variety[same_variety['시점'] < row['시점']]['평균가격(원)'].last_valid_index()
        next_value = same_variety[same_variety['시점'] > row['시점']]['평균가격(원)'].first_valid_index()
        
        if prev_value is not None and next_value is not None:
            return (df.loc[prev_value, '평균가격(원)'] + df.loc[next_value, '평균가격(원)']) / 2
        elif prev_value is not None:
            return df.loc[prev_value, '평균가격(원)']
        elif next_value is not None:
            return df.loc[next_value, '평균가격(원)']

    return row['평균가격(원)']

# 각 작물에 대해 결측치 처리 적용
for crop, varieties in crop_varieties.items():
    if crop == '건고추':
        df.loc[df['품목명'] == crop, '평균가격(원)'] = df[df['품목명'] == crop].apply(fill_missing_chili, axis=1)
    else:
        ratios, base_variety = calculate_ratios(df, crop, varieties)
        df.loc[df['품목명'] == crop, '평균가격(원)'] = df[df['품목명'] == crop].apply(
            lambda row: fill_missing_by_crop(row, crop, varieties, ratios, base_variety), axis=1
        )

print("\n결측치 처리 후:")
print(df['평균가격(원)'].isnull().sum(), "개의 NaN 값")

# 색상 맵핑 (등급별로 다른 색상 지정)
grade_colors = {
    '특': 'red',
    '상': 'blue',
    '상품': 'green',
    '중품': 'purple',
    '중': 'orange',
    '하': 'brown'
}

# 변화 확인을 위한 시각화 함수
def plot_price_trends_comparison(df_original, df_processed):
    for (품목, 품종), group in df_processed.groupby(['품목명', '품종명']):
        for 거래단위, trade_group in group.groupby('거래단위'):
            plt.figure(figsize=(35, 6))
            
            for grade, grade_group in trade_group.groupby('등급'):
                original_group = df_original[(df_original['품목명'] == 품목) & 
                                             (df_original['품종명'] == 품종) & 
                                             (df_original['거래단위'] == 거래단위) & 
                                             (df_original['등급'] == grade)]
                
                plt.plot(original_group['시점'], original_group['평균가격(원)'], 
                         marker='o', linestyle='--', 
                         label=f'{품목}-{품종} ({grade}) 처리 전', 
                         color=grade_colors.get(grade, 'black'))
                
                plt.plot(grade_group['시점'], grade_group['평균가격(원)'], 
                         marker='x', linestyle='-', 
                         label=f'{품목}-{품종} ({grade}) 처리 후', 
                         color=grade_colors.get(grade, 'black'))
            
            plt.title(f'[{품목} - {품종}] 평균가격(원) ({거래단위}) (2018-2021)')
            plt.xlabel('시점')
            plt.ylabel('평균가격(원)')
            
            unique_dates = trade_group['시점'].sort_values().unique()
            plt.xticks(unique_dates, rotation=45)
            
            plt.grid(True)
            plt.legend()
            plt.tight_layout()
            plt.show()

# 그래프 생성
plot_price_trends_comparison(df_original, df)

# 전체 데이터의 결측치 처리 전후 비교
plt.figure(figsize=(20, 10))
plt.scatter(df_original['시점'], df_original['평균가격(원)'], 
            alpha=0.5, label='처리 전', color='blue', marker='o')
plt.scatter(df['시점'], df['평균가격(원)'], 
            alpha=0.5, label='처리 후', color='red', marker='x')
plt.title('전체 데이터의 평균가격 분포 (처리 전/후 비교)')
plt.xlabel('시점')
plt.ylabel('평균가격(원)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '.\\data\\train\\train.csv'

In [ ]:
import pandas as pd

# '시점' 칼럼을 datetime 타입으로 변환
df['시점'] = pd.to_datetime(df['시점'], format='%Y%m%d')

# 상순, 중순, 하순으로 변환하는 함수 정의
def classify_day(date):
    year = date.year
    month = date.month
    day = date.day
    if day <= 10:
        return f"{year}{month:02d}{'상순'}"
    elif day <= 20:
        return f"{year}{month:02d}{'중순'}"
    else:
        return f"{year}{month:02d}{'하순'}"

# '시점' 칼럼에 classify_day 함수 적용
df['시점'] = df['시점'].apply(classify_day)

# 결과 확인
print(df['시점'].head())

In [3]:
# 처리된 결과 저장
df.to_csv('data_preprocessing\train_preprocess/평균.csv', index=False)